## Langchain

LangChain enables building application that connect external sources of data and computation to LLMs.

1.  A simple LLM chain, which just relies on information in the prompt template to respond.
2.  we will build a retrieval chain, which fetches data from a separate database and passes that into the prompt template.
3.  We will then add in chat history, to create a conversation retrieval chain.
4.  Finally, we will build an agent - which utilizes an LLM to determine whether or not it needs to fetch data to answer questions.

#### LLM Chain

https://python.langchain.com/docs/modules/model_io/

In [51]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key="your_api_key")

In [4]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated testing: Langsmith can help create automated test scripts that can be run repeatedly to ensure the functionality of the software is working as expected.\n\n2. Test case generation: Langsmith can assist in generating test cases based on the requirements and specifications of the software, helping to ensure comprehensive test coverage.\n\n3. Test data generation: Langsmith can help create test data that can be used to test different scenarios and edge cases, ensuring the software is robust and resilient.\n\n4. Performance testing: Langsmith can help with performance testing by simulating different load levels and stress testing the software to identify potential bottlenecks and performance issues.\n\n5. Integration testing: Langsmith can assist in testing the integration of different components or systems to ensure they work together seamlessly.\n\nOverall, Langsmith can help streamline the testing proces

In [5]:
# Prompt templates convert raw user input to better input to the LLM.
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
# A simple LLM chain:
chain = prompt | llm

In [7]:
# We can now invoke it and ask the same question.
# The output of a ChatModel (and therefore, of this chain) is a message.
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith is a powerful tool that can greatly assist in testing software applications. Here are some ways in which Langsmith can help with testing:\n\n1. **Automated Testing**: Langsmith can be used to automate the testing of various components of a software application. By writing scripts in Langsmith, testers can automate repetitive tasks such as regression testing, performance testing, and integration testing.\n\n2. **Data Generation**: Langsmith can generate test data for various scenarios, allowing testers to simulate different user inputs and edge cases. This helps in ensuring comprehensive test coverage and the identification of potential issues.\n\n3. **API Testing**: Langsmith can be used to test APIs by sending HTTP requests and verifying the responses. Testers can write scripts to perform API testing and validate the data returned by the API.\n\n4. **Integration Testing**: Langsmith can be used to test the integration of different components of a software 

In [8]:
# A simple output parser to convert the chat message to a string.

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
# We can now add this to the previous chain:
chain = prompt | llm | output_parser

In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can help with testing in several ways:\n\n1. Automated testing: Langsmith can be used to automate the testing of code, helping to identify bugs and issues early in the development process.\n\n2. Test data generation: Langsmith can be used to generate test data to ensure thorough testing of various scenarios and edge cases.\n\n3. Performance testing: Langsmith can be used to simulate different loads and scenarios to test the performance of an application or system.\n\n4. Compatibility testing: Langsmith can be used to test the compatibility of software across different platforms, browsers, and devices.\n\n5. Security testing: Langsmith can be used to simulate various security threats and vulnerabilities to test the security of an application or system.\n\nOverall, Langsmith can be a valuable tool in the testing process, helping to improve the quality and reliability of software products.'

#### Retrieval Chain

To properly answer the original question ("how can langsmith help with testing?"), we need to provide additional context to the LLM. We can do this via retrieval.

Retrieval is useful when you have too much data to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in.

In this process, we will look up relevant documents from a Retriever and then pass them into the prompt.

In this instance we will populate a vector store and use that as a retriever.
https://python.langchain.com/docs/modules/data_connection/vectorstores/

In [11]:
# First, we need to load the data that we want to index.
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [12]:
print(docs)

[Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing ne

In [53]:
# Next, we need to index it into a vectorstore. 
#This requires a few components, namely an embedding model and a vectorstore.

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key="Your_api_key")


In [14]:
# Now, we can use this embedding model to ingest documents into a vectorstore.
# We will use a simple local vectorstore, FAISS, for simplicity'sake: pip install faiss-cpu

# Facebook AI Similarity Search (Faiss)

# Then we can build our index:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
print(documents)
vector = FAISS.from_documents(documents, embeddings)

[Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}), Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how th

In [17]:
print(vector)

In [19]:
# Now that we have this data indexed in a vectorstore, we will create a retrieval chain.
# This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

# set up the chain that takes a question and the retrieved documents and generates an answer.
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [21]:
print(document_chain)

bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='Answer the following question based only on the provided context:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020D2A504BB0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020D2A5067D0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser() config={'run_name': 'stuff_documents_chain'}


In [22]:
# If we wanted to, we could run this ourselves by passing in documents directly:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results.'

In [24]:
# However, we want the documents to first come from the retriever we just set up.
# That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [25]:
# We can now invoke this chain.
# This returns a dictionary - the response from the LLM is in the answer key

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

In [27]:
print(response)

print("-------------------------------")
print(response["answer"])

{'input': 'how can langsmith help with testing?', 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new 

#### Conversation Retrieval Chain

One of the main types of LLM applications that people are building are chat bots.

We can still use the create_retrieval_chain function, but we need to change two things:

1. The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
2. The final LLM chain should likewise take the whole history into account

###### Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query.

In [28]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [29]:
# We can test this out by passing in an instance where the user asks a follow-up question.

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications, we suggest having LangSmith tracing enabled

In [30]:
# Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [31]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug wh

## Agent

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

1. The retriever we just created. This will let it easily answer questions about LangSmith
2. A search tool. This will let it easily answer questions that require up-to-date information.

In [34]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [52]:
import os
os.environ["TAVILY_API_KEY"] = "your_key"


In [40]:
# The search tool that we will use is Tavily
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [42]:
# We can now create a list of the tools we want to work with:
tools = [retriever_tool, search]

In [50]:
import os
os.environ["OPENAI_API_KEY"] = "Your_Api_Key"


In [46]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [47]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phas

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith can help with testing in various ways throughout the application development lifecycle. Here are some key ways LangSmith can assist with testing:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategies, and other parameters for LLM applications.\n\n2. Debugging: LangSmith provides tracing capabilities that allow developers to debug issues by looking through application traces and identifying root causes of problems.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications, helping to ensure the application functions as expected.\n\n4. Comparison View: LangSmith offers a comparison view for test runs, allowing developers to track and diagnose regressions in test scores across multiple revisions of the application.\n\n5. Playground: LangSmith provides a playground environment for rapid iteration and e

In [48]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1714723209, 'localtime': '2024-05-03 1:00'}, 'current': {'last_updated_epoch': 1714722300, 'last_updated': '2024-05-03 00:45', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 20.6, 'wind_kph': 33.1, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1015.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 100, 'feelslike_c': 11.4, 'feelslike_f': 52.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 25.1, 'gust_kph': 40.3}}"}, {'url': 'https://weather.com/weather/tenday/l/

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is as follows:\n- Temperature: 53.1°F (11.7°C)\n- Condition: Overcast\n- Wind: 20.6 mph from the west\n- Humidity: 86%\n- Cloud cover: 100%\n- Feels like: 52.5°F (11.4°C)\n- Visibility: 9.0 miles\n\nIf you need more detailed information or a forecast, feel free to ask!'}

In [49]:
# We can have conversations with it:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for th

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith is a platform that supports LLM application development, monitoring, and testing. Here are some ways LangSmith can help test your LLM applications:\n\n1. Prototyping: LangSmith allows for quick experimentation between prompts, model types, retrieval strategies, and other parameters to understand how the model is performing during the prototyping phase.\n\n2. Debugging: LangSmith provides tracing capabilities to help debug issues in LLM applications by giving clear visibility and debugging information at each step of an LLM sequence.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications, enabling them to evaluate the performance of their applications.\n\n4. Comparison View: LangSmith offers a comparison view for test runs to track and diagnose regressions in t

Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API.